# Exercise: Hyperparameters

what are hyperparemeters.

what we are doing in this Exercise

We have to talk about the importance of cost AND iterations BEFORE starting to talk about learning rate:

Gist is:
High learning rate, may overshoot option weights and increase cost with moe iterations

Low learning rate: typically decrease cost for each iteration but may be too slow

Plot cost as a funciton of iterations


## Preparing data




In [1]:
import pandas as pd



## Summary

Recall what hyperparams are and how they influenced the results.
